# Entrance access analysis

[More about entrance access](/spatial_computing_project_template/A3_Massing/Process/Public%20entrance%20analysis/Public%20entrance%20analysis/)

In this notebook, we will analyse the access to seven different entrances. This is usefull for placing specific agents close to a specific entrance

## 0. Initialization

### 0.1. Load required libraries

In [ ]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import networkx as nx
import pickle
np.random.seed(0)

### 0.2. Define the Neighborhood (Stencil)

In [ ]:
# creating neighborhood definition
stencil = tg.create_stencil("von_neumann", 1, 1)
# setting the center to zero
stencil.set_index([0,0,0], 0)

### 0.3. Load the envelope lattice as the avialbility lattice

In [ ]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/final_envelope_new.csv')
avail_lattice = tg.lattice_from_csv(lattice_path)
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice), avail_lattice)

## 1. Distance Field Construction

### 1.1. Extract the connectivity graph from the lattice based on the defined stencil

In [ ]:
# find the number of all voxels
vox_count = avail_lattice.size 

# initialize the adjacency matrix
adj_mtrx = np.zeros((vox_count,vox_count))

# Finding the index of the available voxels in avail_lattice
avail_index = np.array(np.where(avail_lattice == 1)).T

# fill the adjacency matrix using the list of all neighbours
for vox_loc in avail_index:
    # find the 1D id
    vox_id = np.ravel_multi_index(vox_loc, avail_lattice.shape)
    # retrieve the list of neighbours of the voxel based on the stencil
    vox_neighs = avail_lattice.find_neighbours_masked(stencil, loc = vox_loc)
    # iterating over the neighbours
    for neigh in vox_neighs:
        # setting the entry to one
        adj_mtrx[vox_id, neigh] = 1.0

# construct the graph 
g = nx.from_numpy_array(adj_mtrx)

### 1.2. Compute distances on the graph

In [ ]:
# compute the distance of all voxels to all voxels using floyd warshal algorithm
# dist_mtrx = nx.floyd_warshall_numpy(g)
dist_mtrx_loaded = pickle.load( open( "../data/dist_mtrx_final.p", "rb" ) )

### 1.3. Select the entrance voxel

In [ ]:
p = pv.Plotter(notebook=True)

# initialize the selection lattice
base_lattice = avail_lattice * 0 - 1

# init base flat
base_flat = base_lattice.flatten().astype(int)

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):
    i = int(value)
    # init base flat
    base_flat = base_lattice.flatten().astype(int)
    base_flat = base_flat * 0 - 1
    base_flat[i] = 0 
    base_new = base_flat.reshape(base_lattice.shape)
    # Add the data values to the cell data
    grid.cell_arrays["Selection"] = base_new.flatten(order="F").astype(int) # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([-0.9, 0.9])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)

    return

p.add_slider_widget(create_mesh, [5000, 6000], title='1D Index', value=0, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))
p.show(use_ipyvtk=True)

### 1.4. Construct Distance to 1st Entrance Lattice 

In [ ]:
# this cel is needed when we want to combine enntrance lattices. Here we define the first entrance
"""# select the corresponding row in the matrix
ent_1_dist = dist_mtrx_loaded[138]

# find the maximum valid value
max_valid = np.ma.masked_invalid(ent_1_dist).max()

# set the infinities to one more than the maximum valid values
ent_1_dist[ent_1_dist == np.inf] = max_valid + 1"""

### 1.5. Construct Distance to 2nd Entrance Lattice 

In [ ]:
# this cel is needed when we want to combine enntrance lattices. Here we define the second entrance
"""# select the corresponding row in the matrix
ent_2_dist = dist_mtrx_loaded[313]

# find the maximum valid value
max_valid = np.ma.masked_invalid(ent_2_dist).max()

# set the infinities to one more than the maximum valid values
ent_2_dist[ent_2_dist == np.inf] = max_valid + 1"""

### 1.6. Combining different distance fields

In [ ]:
# here the entrance lattices are combined
"""# finding the minimum distance between two entrance 
# fields (aka finding the closest entrance and replacing 
# the distance with the distance to that entrance)
ent_dist_temp1 = np.minimum(ent_1_dist, ent_2_dist)
ent_dist_temp2 = np.minimum(ent_dist_temp1, ent_3_dist)
ent_dist_temp3 = np.minimum(ent_dist_temp2, ent_4_dist)
ent_dist_temp4 = np.minimum(ent_dist_temp3, ent_5_dist)
ent_dist = np.minimum(ent_dist_temp4, ent_6_dist)

# mapping the values from (0, max) to (1, 0)
ent_flat = 1 - ent_dist / np.max(ent_dist)

# constructing the lattice
ent_acc_lattice = tg.to_lattice(ent_flat.reshape(avail_lattice.shape), avail_lattice)
"""
# select the corresponding row in the matrix. This is done when only one entrance is used in the distance lattice
ent_dist = dist_mtrx_loaded[5786]

# find the maximum valid value
max_valid = np.ma.masked_invalid(ent_dist).max()
# set the infinities to one more than the maximum valid values
ent_dist[ent_dist == np.inf] = max_valid + 1

# mapping the values from (0, max) to (1, 0)
ent_flat = 1 - ent_dist / np.max(ent_dist)

# constructing the lattice
ent_acc_lattice = tg.to_lattice(ent_flat.reshape(avail_lattice.shape), avail_lattice)

### 1.7. Visualize the distance lattice

In [ ]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# load the mesh from file
context_path = os.path.relpath('../data/immediate_context.obj')
context_mesh = tm.load(context_path)

# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = ent_acc_lattice.shape
# The bottom left corner of the data set
grid.origin = ent_acc_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = ent_acc_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Entrance Access"] = ent_acc_lattice.flatten(order="F")  # Flatten the Lattice

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')
    
# adding the volume
opacity = np.array([0.0,0.6,0.6,0.6,0.6,0.6,0.6]) * 0.6
p.add_volume(grid, cmap="coolwarm", clim=[0.0, 1.0] ,opacity=opacity)

# plotting
p.show(use_ipyvtk=True)

### 1.6. Save Entrance Access Lattice to CSV

In [ ]:
# save the entrance access latice to csv
csv_path = os.path.relpath('../data/ent2_access.csv')
ent_acc_lattice.to_csv(csv_path)

### Credits

In [ ]:
__author__ = "Shervin Azadi and Pirouz Nourian"
__changes_made_by__ "Xam Adan"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/frankvahstal/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on MCDA and Path Finding for Generative Spatial Relations"